# RAG + MLflow Demo Notebook

Dieses Notebook demonstriert:
- Erzeugen von Embeddings mit `sentence-transformers`
- Indexieren mit `faiss`
- Retrieval für eine Anfrage
- Tracking von Embedding-Modell, Embeddings und FAISS-Index mit `mlflow`

Ausführungsanleitung:
1. Installiere Abhängigkeiten: `pip install -r requirements.txt`
2. Starte das Notebook und führe die Zellen nacheinander aus.
3. Optional: `mlflow ui --backend-store-uri ./mlruns` um Runs zu inspizieren.


## 1) Imports und Beispieldokumente

In [ ]:

# Beispiel-Dokumente
documents = [
    "The Port of Hamburg is one of the largest ports in Europe and handles millions of TEU annually.",
    "Container throughput can be forecasted using time-series models and operational features like arrival rates and berth availability.",
    "Predictive maintenance for cranes reduces downtime by combining sensor data with failure logs and scheduled inspections.",
    "RAG systems use dense vector retrieval (embeddings) to fetch supporting documents, then condition an LLM on the retrieved context."
]
len(documents)


## 2) Install Hinweis

Install required packages: `pip install -r requirements.txt`

## 3) Embeddings erstellen

In [ ]:

from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(documents, convert_to_numpy=True)
print('Embeddings shape:', embeddings.shape)


## 4) FAISS Index erstellen

In [ ]:

import faiss
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)
print('Indexed vectors:', index.ntotal)


## 5) Retrieval-Funktion

In [ ]:

def retrieve(query, k=2):
    q_emb = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(q_emb, k)
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        results.append({'doc': documents[idx], 'score': float(dist)})
    return results

retrieve('How to forecast container throughput?', k=2)


## 6) MLflow Integration — lokal

In [ ]:

# MLflow Beispiel: lokal tracken (./mlruns)
import mlflow
import json
import faiss

mlflow.set_tracking_uri('file://' + './mlruns')
with mlflow.start_run(run_name='notebook_rag_demo'):
    mlflow.log_param('embedding_model', 'all-MiniLM-L6-v2')
    # Speichere Dokumente
    with open('documents.json','w',encoding='utf-8') as f:
        json.dump(documents, f, ensure_ascii=False, indent=2)
    mlflow.log_artifact('documents.json', artifact_path='rag_docs')
    # Embeddings als numpy speichern und loggen
    np.save('embeddings.npy', embeddings)
    mlflow.log_artifact('embeddings.npy', artifact_path='embeddings')
    # FAISS Index speichern
    faiss.write_index(index, 'faiss.index')
    mlflow.log_artifact('faiss.index', artifact_path='faiss_index')
    # Beispiel-Retrieval loggen
    query = 'Welche Methoden eignen sich zur Vorhersage von Ankunftszeiten im Hafen?'
    results = retrieve(query, k=2)
    with open('retrieval.json','w',encoding='utf-8') as f:
        json.dump({'query': query, 'results': results}, f, ensure_ascii=False, indent=2)
    mlflow.log_artifact('retrieval.json', artifact_path='retrievals')
    # Log distances as metrics
    for i, r in enumerate(results):
        mlflow.log_metric(f'dist_{i}', r['score'])
    print('MLflow run logged — öffne mlflow ui mit: mlflow ui --backend-store-uri ./mlruns')


## 7) Prompt composition (ohne LLM-Aufruf)

In [ ]:

retrieved = retrieve('What methods to forecast arrivals in a port?', k=2)
context = '\n\n'.join([r['doc'] for r in retrieved])
user_question = 'Welche Methoden eignen sich zur Vorhersage von Ankunftszeiten im Hafen?'

prompt = f"""You are an expert on port logistics. Use the following context to answer the question.

Context:
{context}

Question:
{user_question}

Answer concisely in German and mention key features you would use for forecasting.
"""

print(prompt)
